In [2]:
import os
import joblib
import math
import warnings
import gc
warnings.filterwarnings('ignore')
from tqdm import tqdm
import pickle

import bloscpack as bp

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import NuSVC
from sklearn.model_selection import GroupKFold, StratifiedKFold, train_test_split, RepeatedStratifiedKFold
from sklearn import metrics

import lightgbm as lgb

with open('../input/batch_ids_trn.pkl', 'rb') as f:
    batch_id_trn = pickle.load(f)
with open('../input/batch_ids_tst.pkl', 'rb') as f:
    batch_id_tst = pickle.load(f)

In [3]:
trn_dat = bp.unpack_ndarray_from_file(os.path.join('../input/feats_tblr', 'trn_dat_all_origv2_w500.bp'))
trn_lbl = pd.read_pickle('../input/trn_lbl_orig_v3_all.pkl')['open_channels'].values

In [19]:
# strat_lbl = pd.read_pickle('../input/tblr_data_stratification_group.pkl')
# strat_bch = strat_lbl.index // 500000
# strat_frm = pd.DataFrame({'strat_signal': strat_lbl, 'batch': strat_bch})
# strat_lbl = strat_frm.groupby('batch')['strat_signal'].transform(lambda x: pd.qcut(x, 100, labels=False).fillna(999)).values.astype(int)

new_lbl = [
    str(a) + '_' + str(b) for a, b, c in zip(
        strat_lbl.astype('uint32'),
        np.concatenate([np.ones(500000).astype('uint32') * i for i in range(10)]),
        trn_lbl.astype('uint32'),
    )
]
unq_l = np.unique(new_lbl)
lbl_map = {str_l: i for str_l, i in zip(unq_l, np.arange(len(unq_l)))}
new_lbl = [lbl_map[s] for s in new_lbl]

In [30]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [31]:
for fold, (trn_ndcs, vld_ndcs) in enumerate(kf.split(trn_dat, new_lbl)):
    if fold == 0:
        x_trn, x_vld = trn_dat[trn_ndcs], trn_dat[vld_ndcs]
        y_trn, y_vld = trn_lbl[trn_ndcs], trn_lbl[vld_ndcs]
        del trn_dat, trn_lbl
        break
    #trn_set = lgb.Dataset(x_trn, y_trn)
    #vld_set = lgb.Dataset(x_vld, y_vld)

In [38]:
x_trn.min()

-525.2337494216519

In [ ]:
model = NuSVC(nu=0.05)
model.fit(X=x_trn, y=y_trn)

In [ ]:
# from datetime import datetime
print(datetime.now())

In [ ]:
1

In [22]:
joblib.dump(model, './saved_models/lgbm_cls_feats_origv2_trgtenc_fld{:d}.pkl'.format(fold))

['./saved_models/lgbm_cls_feats_origv2_trgtenc_fld0.pkl']